In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd
from collections import Counter
from itertools import islice 
import matplotlib.pyplot as plt
import folium
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Exploring the Data File

Let us take a look at our data. Though in kaggle dataset, the description of rows and columns can be found, it is a good practice to have a glimpse of the data using `head()` method. It shows the first five rows of the data file.

In [ ]:
data_file = pd.read_csv('../input/bays-and-gulfs-of-the-world/Bays and Gulfs.csv')
data_file.head()

The `read_csv()` method returns a DataFrame. To investigate in details about the DataFrame, we can use the `info()` method. This will show us some important aspects of the data e.g. data type, non-null values,  memory usage etc. In this case, we have 4 objects (strings) and 2 float64 type data in total 6 columns. The most important thing  to notice here is that there is no null values. Therefore, investigating with `info()` will make our life a bit easier.

In [ ]:
data_file.info()

## Visualizing the Geospatial Data

Firstly, we can see the world distribution of Gulfs and Bays. The Geopandas package is used here. We convert the DataFrame to a GeoDataFrame. We also need a world map shapefile. The Geopandas can read .shp file using the `read_file()` method. 

In [ ]:
world_map = gpd.read_file('../input/shpfile')
bays_gulfs = gpd.GeoDataFrame(data_file, geometry= gpd.points_from_xy(data_file.Longitude,data_file.Latitude))
base = world_map.plot(color='#B3BA0A', edgecolor='#B3BA0A',figsize=(24,24))
bays_gulfs.plot(ax = base,color='#085FB8',marker='o', markersize=18)

The blue dots are the locations of gulfs and bays. Africa seems to have the least ones. It is clear that Europe and Asia have the most inlets. Australia, Newzealand and Papua New Guienea are almost completely surrounded by bays. We can make this static map more interactive using `folium`.

## Folium 

The folium package makes it easy to create an interactive web map. We can not only show the locations of our inlets but also display various attributes.

In [ ]:
world_map = folium.Map(location=[10.5,5.10], zoom_start=2,  tiles='openstreetmap')
def plotBay(d_f):
    folium.Marker(location=[d_f.Latitude, d_f.Longitude], popup=d_f['Gulf/Bay']+' in\n'+ d_f['Ocean/Sea'],icon=folium.Icon( icon='info-sign')).add_to(world_map)
data_file.apply(plotBay,axis=1)
world_map
        

If you click on the markers, you can see the name of the gulf/bay and which sea or ocean it belongs to. The `popup` parameter makes it possible.

## Doing a bit Statistics

Lastly, let us get our eyes on the number of bays and their surrounding continents. Here we will use the `matplotlib.pyplot` package and create a bar chart. We will try to find which continent has the most gulfs/bays.

In [ ]:
dict_continent = Counter(data_file['Continent'])
#print(dict_continent)

plt.figure(figsize=(21,21))
plt.bar(dict_continent.keys(),dict_continent.values())
plt.title('Number of Bays and Gulfs Vs Continents',pad=15,fontdict={'fontsize': 24,'fontweight':'bold'})
plt.xlabel(xlabel="Continent",labelpad=15, fontdict={'fontsize': 18,'fontweight':'bold'})
plt.ylabel(ylabel="Number of Bays/Gulfs",labelpad=15,fontdict={'fontsize': 18,'fontweight':'bold'})
plt.show()

It is evident that most of the bays and gulfs are assosicated with Asia. The conjugate continents share a certain number of inlets but the number is small compared to individual continents. As we get back to our map, we can see that Asia and Oceania do have the dense distribution of bays and gulfs. Interestingly, the North America and Europe have almost equal number of inlets. We did not get this picture clearly from the map. However, now we have got a descriptive view of our geospatial perspective. Therefore, getting insights from both perspectives can be helpful in clearly understanding a spatial distribution.